# Get trending campaigns from GiveSendGo

In [257]:
import pandas as pd
import cloudscraper
import json
import requests
from time import sleep
from random import randint
from bs4 import BeautifulSoup
import numpy as np

In [209]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Launch our scraper object

In [237]:
headers = {
    'User-Agent':
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept-Encoding': 'gzip, deflate'
}

session = requests.session()
session = cloudscraper.create_scraper(session)

In [238]:
campaign_list = []

for x in range(1,101):
    url = 'https://givesendgo.com/campaigns?page='
    p = session.get(url+str(x), headers=headers).text
    soup = BeautifulSoup(p, 'html.parser')
    campaigns = soup.find_all('div', class_='main_camps')
    
    for campaign in campaigns:
        name = campaign.find("div", {'class':"camp_title"}).text.strip('\n')
        text = campaign.find("p").text.strip('\n')
        group = campaign.find("div", {'class':"fund_by"}).text.strip().strip('\n').replace('\t', '').replace('by \n                         \n                         ', "")
        donors = campaign.find("div", {'class':"givers-heart"}).text.replace('givers', '')
        try:
            raised = campaign.find("div", {'class':"price-range-trade"}).text.strip('\n')
        except: 
            pass

        campaign_details = {
            "name": name,
            'text': text,
            "group": group,
            "donor_count": donors,
            "raised": raised
        }

        campaign_list.append(campaign_details)

In [265]:
src = pd.DataFrame(campaign_list)

In [266]:
src[['total', "goal"]] = src['raised'].str.replace('\n', "", regex=False).str.strip().str.split('Raised', expand=True)

In [267]:
src['currency'] = src['total'].str.replace('\d+', '', regex=True).str.replace('$', "", regex=False).str.replace(" .", "", regex=False)

In [268]:
src['funding_total'] = src['total'].str.replace("[r'\D+']", "", regex=True)

In [269]:
src['funding_goal'] = src['goal'].str.replace("[r'\D+']", "", regex=True)

In [270]:
src.drop(['raised', 'total'], axis=1, inplace=True)

In [286]:
df = src[['name', 'group', 'text', 'donor_count', "funding_total", 'funding_goal', 'currency']].reset_index(drop=True).copy()

In [287]:
df['text'] = df['text'].str.replace('\n ', "")

In [288]:
df['funding_total'] = df['funding_total'].replace(r'^\s*$', np.nan, regex=True).astype(float)

In [289]:
df.sort_values('funding_total', ascending=False).head(10)

,name,group,text,donor_count,funding_total,funding_goal,currency
0,Freedom Convoy 2022,Freedom 2022 Human Rights,"To our Fellow Canadians, the time for political ov...",90867,8237143.0,16000000,USD
310,Help Aaron Beat Brain Cancer,Jennifer Robinson,"My husband, Aaron, is in the fight of his life. On...",82,925100.0,17000,USD
2,Adopt a trucker,Chris Garrah,Please register your offer at http://register.adop...,7341,676184.0,,CAD
259,Transsplendor support fund,Alexander Cargill,I'm heading to Ottawa to report on events of the b...,81,395677.0,2000,CAD
97,Defeat the Mandates,Pandemic Health Alliance,"United We Stand, In\r\nPeace We MarchSunday, January...",2953,284327.0,500000,USD
325,We The People Reunion Ky Tornado Fund,Rebecca Jeffries,1776 Events LLC as We The People Reunion are raisi...,2616,208896.0,,USD
308,VCA On The Move,Victory Christian Academy,The news Victory Christian Academy has purchased a...,16,193648.0,400000,USD
1,Legal Defense Fund,John Eastman,John Eastman served as President Trump's lawye...,2454,172500.0,200000,USD
38,Medical Relief Fund,Anne Quiner,"\r\nOn Tuesday, October 30th, Scott Quiner was test...",1984,147569.0,100000,USD
298,Keep Independent Media Alive,Michael Rivero,"Like the Canadian Truck convoy, I was kicked off o...",14,91400.0,3000,USD


In [290]:
df[df['name'].str.lower().str.contains("truck|convoy")].sort_values('funding_total', ascending=False)

,name,group,text,donor_count,funding_total,funding_goal,currency
0,Freedom Convoy 2022,Freedom 2022 Human Rights,"To our Fellow Canadians, the time for political ov...",90867,8237143.0,16000000,USD
2,Adopt a trucker,Chris Garrah,Please register your offer at http://register.adop...,7341,676184.0,,CAD
166,Freedom Convoy to DC 2022,Daniel Greig,Up in Canada we\r\nstood up to the government and ca...,1323,87586.0,250000,CAD
48,Freedom convoy truck fuel and food donate,Jay L.,Looking for donations for fuel for truckers and an...,133,14474.0,,CAD
89,Help us continue to feed the Truckers,AlphaCater Inc,"Hello,We are currently feeding this amazing trucke...",158,13366.0,5000,CAD
138,Freedom Convoy US,Freedom Convoy US,This is the official fundraiser for the Convoy to ...,159,8368.0,5000,USD
61,BC Freedom Convoy 2022,Christian McCay,Now Is the time to get back our freedoms and right...,21,7006.0,,CAD
287,FREEDOM CONVOY 2022 HELPERS,Help3r,"Founder of Help3r.com, The new Help Marketplace in...",69,5246.0,10000,CAD
67,Convoy to DC,Denby Morgan,American truckers and citizens stand in solidarity...,83,3528.0,,USD
150,Send Us Back To Ottawa To Cover The Freedom Convoy,Near Death Experiment,Help send us back to Ottawa to continue documentin...,24,3340.0,5000,CAD


In [291]:
len(df[df['name'].str.lower().str.contains("truck|convoy")])

16